# Upscaling PPFI from LSOA to LAD (and preparing LAD 2024 linkage)


## LAD-level PPFI (Population-weighted upscaling)

To create a LAD 2024 version of PPFI that is consistent with the original methodology:

1. Attach LSOA 2021 population (Census 2021 usual residents) to the LSOA indicator table.
2. Aggregate indicators from LSOA to LAD 2024 using a population-weighted mean.
3. Re-run the full PPFI scoring pipeline at LAD level (using the aggregated indicators):
   - indicator orientation (higher = higher priority)
   - within-country ranking
   - normal-score transformation
   - domain construction
   - exponential transformation
   - combined score + within-country deciles

### How to interpret the LAD measure
Because indicators are population-weighted when aggregating to LAD, the LAD scores represent the average indicator conditions experienced by residents within each LAD (rather than the “average LSOA”). This makes the LAD-level index more comparable for people-focused interpretation, while remaining methodologically aligned with the original PPFI steps.

### Notes:
- The LAD output is limited to LADs present in the LSOA to LAD 2024 best-fit lookup (England & Wales).
- Domain construction and weighting follow the original PPFI implementation used in this notebook.

In [1]:
# import required libraries
import pandas as pd
import numpy as np
from scipy.stats import norm
import scipy as sp
from datetime import datetime

## Load inputs and build geography lookups

We load:
- Raw PPFI indicators at LSOA 2021 (one row per LSOA with the indicator columns).
- A lookup mapping LSOA 2021 > MSOA 2021 > LAD (using LAD codes provided in the OA>LSOA>MSOA>LAD lookup).
- A best-fit lookup mapping LSOA 2021 > LAD 2024 (via ward best-fit).

We then create a single lookup table that attaches both:
- ladcd (from the older lookup), and
- lad24cd (for LAD 2024),
to each lsoa21cd

In [ ]:
# import ppfi raw scores and lookup table
ppfi_raw = pd.read_csv('full_indicators_corrected_hsv.csv')
lookup_lsoa_to_lad21 = pd.read_csv('Output_Area_to_Lower_layer_Super_Output_Area_to_Middle_layer_Super_Output_Area_to_Local_Authority_District_(December_2021)_Lookup_in_England_and_Wales_v3.csv')
lookup_lsoa_to_lad24 = pd.read_csv('LSOA_(2021)_to_Electoral_Ward_(2024)_to_LAD_(2024)_Best_Fit_Lookup_in_EW.csv')

/var/folders/rc/vyj9c1152wd3y84hd9qp19cr0000gn/T/ipykernel_45950/2322392278.py:3: DtypeWarning: Columns (3,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup_lsoa_to_lad21 = pd.read_csv('Output_Area_to_Lower_layer_Super_Output_Area_to_Middle_layer_Super_Output_Area_to_Local_Authority_District_(December_2021)_Lookup_in_England_and_Wales_v3.csv')


In [3]:
lookup_lsoa_to_lad21.columns

Index(['OA21CD', 'LSOA21CD', 'LSOA21NM', 'LSOA21NMW', 'MSOA21CD', 'MSOA21NM',
       'MSOA21NMW', 'LAD22CD', 'LAD22NM', 'LAD22NMW', 'ObjectId'],
      dtype='object')

In [4]:
# builing lookup from lsoa > lad
# first we will change the column names
lookup_lsoa_to_lad21 = lookup_lsoa_to_lad21.rename(columns={'LSOA21CD':'lsoa21cd','LAD22CD':'ladcd'})[['lsoa21cd','ladcd']].drop_duplicates()

In [5]:
#lsoa 2021 to lad 2024 for imd comparison later
lookup_lsoa_to_lad24 = lookup_lsoa_to_lad24.rename(columns={'LSOA21CD':'lsoa21cd','LAD24CD':'lad24cd'})[['lsoa21cd','lad24cd']].drop_duplicates()

In [ ]:
lookup_final = lookup_lsoa_to_lad21.merge(lookup_lsoa_to_lad24,on='lsoa21cd',how='left')

We merge the final lookup onto the raw PPFI indicator dataset so each LSOA row has:
- its indicator values, and
- its LAD identifiers (ladcd and lad24cd)

This provides:
- an LSOA table to recompute PPFI scores, and
- the LAD 2024 codes needed to aggregate indicators to LAD level later.

In [7]:
# merge lookup with ppfi
ppfi_lookup = ppfi_raw.merge(lookup_final, on='lsoa21cd', how='left')
ppfi_lsoa = ppfi_lookup.set_index('lsoa21cd')

The following code was copied over from the original PPFI creation.

In [8]:
# copy over indicator list from original ppfi creation
indicator_cols = [
    # Proximity to and density of retail facilities
    'supermarket_1kmcount', 
    'supermarket_distance', 
    
    # Transport to and accessibility of grocery retail facilities
    'AccessibilityViaPublicTransport', 
    'AverageTravelDistance', 
    
    # E-commerce access
    'online_propensity', 
    'OnlineGroceryAvailability', 
    
    # Neighbourhood socio-economic and demographic
    'no_car_indicator', 
    'inc_dep_indicator', 
    
    # Proximity to and density of non-supermarket food provision
    'nonsupermarket_distance',
    'nonsupermarket_1kmcount',
    'market_1km_count', 
    'market_distance', 
    
    # Food for families
    'households_children_UC_percent',
    'healthy_start_voucher_uptake', 
    'foodbank_distance', 
    
    #Fuel poverty pressures
    'fuel_poverty_pct',
    'prepayment_prevalence']

In [9]:
# copied from original ppfi weighting/creation code
priority_places = ppfi_lsoa[indicator_cols].copy()

priority_places = priority_places.drop_duplicates()

# The first task is to orient each indicator in the correct direction
# i.e. so that high values correspond to higher priority places
priority_places = pd.concat([1 * priority_places[[
                    'supermarket_distance', 
                    'AccessibilityViaPublicTransport', 
                    'AverageTravelDistance', 
                    'no_car_indicator', 
                    'inc_dep_indicator', 
                    'market_distance', 
                    'fuel_poverty_pct', 
                    'prepayment_prevalence', 
                    'nonsupermarket_distance', 
                    'households_children_UC_percent',
                    'healthy_start_voucher_uptake']], 
                  -1 * priority_places[[
                      'foodbank_distance', 
                      'supermarket_1kmcount', 
                      'online_propensity', 
                      'OnlineGroceryAvailability', 
                      'market_1km_count', 
                      'nonsupermarket_1kmcount']]], axis=1)

# Find our country-level denominators


priority_places['country'] = priority_places.index.str[0]
country_counts = priority_places.reset_index(names='index').groupby('country')['index'].count()
priority_places = priority_places.merge(country_counts, left_on='country', right_index=True, how='inner')
priority_places.rename({'index': 'country_denominator'}, inplace=True, axis=1)


# Perform ranking of each indicator
priority_places.fillna(0, inplace=True)
priority_places_ranked = priority_places.groupby('country').rank(method='min', ascending=False).astype(int)

for c in priority_places_ranked[indicator_cols].columns: 
    priority_places_ranked[c] = (priority_places_ranked[c] - 0.5) / priority_places['country_denominator']
    priority_places_ranked[c] = sp.stats.norm.ppf(priority_places_ranked[c],loc=0,scale=1)
    
priority_places_ranked['country'] = priority_places_ranked.index.str[0]

#Combine transformed indicators into domains
priority_places_ranked['domain_supermarket_proximity'] = 0.5 * priority_places_ranked[['supermarket_distance', 'supermarket_1kmcount']].sum(axis=1)
priority_places_ranked['domain_supermarket_accessibility'] = 0.5 * priority_places_ranked[['AccessibilityViaPublicTransport', 'AverageTravelDistance']].sum(axis=1)
priority_places_ranked['domain_ecommerce_access'] = 0.5 * priority_places_ranked[['online_propensity', 'OnlineGroceryAvailability']].sum(axis=1)
priority_places_ranked['domain_socio_demographic'] = (1./2.) * priority_places_ranked[[ 'no_car_indicator', 'inc_dep_indicator']].sum(axis=1)
priority_places_ranked['domain_nonsupermarket_proximity'] = (1./4.) * priority_places_ranked[['nonsupermarket_distance','nonsupermarket_1kmcount','market_1km_count', 'market_distance']].sum(axis=1)
priority_places_ranked['domain_food_for_families'] = (1./4.) * priority_places_ranked[['foodbank_distance', 'healthy_start_voucher_uptake', 'households_children_UC_percent']].sum(axis=1)
priority_places_ranked['domain_fuel_poverty'] = 0.5 * priority_places_ranked[['fuel_poverty_pct','prepayment_prevalence']].sum(axis=1)

domain_columns = ['domain_supermarket_proximity', 
                  'domain_supermarket_accessibility', 
                  'domain_ecommerce_access', 
                  'domain_socio_demographic', 
                  'domain_nonsupermarket_proximity', 
                  'domain_food_for_families', 
                  'domain_fuel_poverty']

# Rank the domains
priority_places_domains = priority_places_ranked[domain_columns + ['country']].groupby('country').rank(method='min').astype(int)
priority_places_domains['country'] = priority_places_domains.index.str[0]

priority_places_domains = priority_places_domains.merge(country_counts, left_on='country', right_index=True, how='inner')
priority_places_domains.rename({'index': 'country_denominator'}, inplace=True, axis=1)

priority_places_domains_normalised = pd.DataFrame(columns=priority_places_domains[domain_columns].columns)
for c in priority_places_domains[domain_columns].columns:
    priority_places_domains_normalised[c] = -23 * np.log(1 - (priority_places_domains[c] / priority_places_domains['country_denominator']) * (1 - np.exp(- 100 / 23)))

priority_places_domains['combined'] = (1./8.) * priority_places_domains_normalised['domain_supermarket_proximity'] + \
(1./8.) * priority_places_domains_normalised['domain_supermarket_accessibility'] + \
(1./8.) * priority_places_domains_normalised['domain_ecommerce_access'] + \
(1./8.) * priority_places_domains_normalised['domain_nonsupermarket_proximity'] + \
(1./6.) * priority_places_domains_normalised['domain_socio_demographic'] + \
(1./6.) * priority_places_domains_normalised['domain_food_for_families'] + \
(1./6.) * priority_places_domains_normalised['domain_fuel_poverty']

priority_places_domains['combined'] = priority_places_domains[['country', 'combined']].groupby('country').rank(method='min').astype(int)

priority_places_deciles = priority_places_domains.copy()
for country in ['E', 'S', 'W', '9']:
    for col in domain_columns + ['combined']:
        if country == '9' and col in ['domain_ecommerce_access', 'domain_supermarket_accessibility', 'domain_fuel_poverty']:
            priority_places_deciles.loc[priority_places_deciles['country']==country, col] = 0
        else:
            priority_places_deciles.loc[priority_places_deciles['country']==country, col] = pd.to_numeric(pd.qcut(priority_places_domains.loc[priority_places_deciles['country']==country, col], 10, duplicates='drop', labels=range(1,11))).astype('int32')
            
priority_places_full = priority_places_domains.merge(priority_places_deciles, left_index=True, right_index=True, suffixes=('', '_decile'))
priority_places_full.drop(['country_decile', 'country_denominator_decile'], axis=1, inplace=True)           

priority_places_full.loc[priority_places_full.index.str.startswith('9'), 
                        ['domain_supermarket_accessibility', 
                         'domain_ecommerce_access', 
                         'domain_fuel_poverty', 
                         'domain_supermarket_accessibility_decile', 
                         'domain_ecommerce_access_decile', 
                         'domain_fuel_poverty_decile']] = pd.NA

# Rename decile columns to align with original field names
priority_places_full.rename({'domain_supermarket_proximity_decile': 'pp_dec_domain_supermarket_proximity',
                             'domain_supermarket_accessibility_decile': 'pp_dec_domain_supermarket_accessibility',
                             'domain_ecommerce_access_decile': 'pp_dec_domain_ecommerce_access',
                             'domain_socio_demographic_decile': 'pp_dec_domain_socio_demographic',
                             'domain_nonsupermarket_proximity_decile': 'pp_dec_domain_nonsupermarket_proximity',
                             'domain_food_for_families_decile': 'pp_dec_domain_food_for_families',
                             'domain_fuel_poverty_decile': 'pp_dec_domain_fuel_poverty',
                             'combined_decile': 'pp_dec_combined'}, inplace=True, axis=1)

priority_places_full.loc[priority_places_full.index.str.startswith('9'), ['country']] = 'NI'


In [10]:
# we dont need to export this dataset as we will use the final version available on the HASP website

We next upscale the PPFI indicators from LSOA to LAD 2024 and recompute the index at LAD level. To create the LAD-level PPFI:

- Aggregate indicators from LSOA to LAD 2024 using population-weighted means: for each lad24cd, indicator values are aggregated across constituent LSOAs using Census 2021 usual resident population as weights. This ensures that larger LSOAs contribute proportionally more to the LAD-level indicator values.

- Re-run the full PPFI scoring methodology at LAD level. The aggregated indicators are processed using the same steps as the LSOA-level PPFI: indicator orientation, within-country ranking, normal-score transformation, domain aggregation, exponential transformation, combined score calculation, and decile creation.

This produces a LAD-level analogue of PPFI that is methodologically consistent with the LSOA version, while reflecting the average indicator conditions experienced by residents within each LAD.

In [ ]:
# population weighting data 

pop_path = 'TS001-Number-Of-Usual-Residents-In-Households-And-Communal-Establishments-2021-lsoa-ONS.xlsx'
pop_raw = pd.read_excel(pop_path, sheet_name='Dataset')

pop_lsoa = (
    pop_raw
    .groupby('Lower Layer Super Output Areas Code', as_index=False)['Observation']
    .sum()
    .rename(columns={
        'Lower Layer Super Output Areas Code': 'lsoa21cd',
        'Observation': 'pop_2021'
    })
)

In [ ]:
# merge population onto ppfi_lookup
ppfi_lookup = ppfi_raw.merge(lookup_final, on='lsoa21cd', how='left')
ppfi_lookup= ppfi_lookup.merge(pop_lsoa, on='lsoa21cd', how='left')

In [ ]:
#population-weighted LAD aggregation
df_temp= ppfi_lookup.dropna(subset=['lad24cd']).copy()

# weighted numerator: sum(x * pop)
weighted_sum= (
    df_temp[indicator_cols]
    .multiply(df_temp['pop_2021'], axis=0)
    .groupby(df_temp["lad24cd"])
    .sum())

# denominator: sum(pop)
pop_sum = df_temp.groupby("lad24cd")["pop_2021"].sum()

# weighted mean
df_lad = weighted_sum.div(pop_sum, axis=0)
df_lad.index.name = "lad24cd"

In [14]:
# copied from original ppfi creation, this time with LAD

priority_places = df_lad[indicator_cols].copy()
priority_places = priority_places.drop_duplicates()

# The first task is to orient each indicator in the correct direction
priority_places = pd.concat([
    1 * priority_places[['supermarket_distance', 'AccessibilityViaPublicTransport',
         'AverageTravelDistance', 'no_car_indicator', 'inc_dep_indicator',
         'market_distance', 'fuel_poverty_pct', 'prepayment_prevalence',
         'nonsupermarket_distance', 'households_children_UC_percent',
         'healthy_start_voucher_uptake']],
    -1 * priority_places[['foodbank_distance', 'supermarket_1kmcount', 'online_propensity',
         'OnlineGroceryAvailability', 'market_1km_count',
         'nonsupermarket_1kmcount']]], axis=1)

# country field
priority_places['country'] = priority_places.index.str[0]
country_counts = priority_places.reset_index(names='index').groupby('country')['index'].count()
priority_places = priority_places.merge(country_counts, left_on='country', right_index=True, how='inner')
priority_places.rename({'index': 'country_denominator'}, axis=1, inplace=True)

# ranking indicators
priority_places.fillna(0, inplace=True)
priority_places_ranked = priority_places.groupby('country').rank(method='min', ascending=False).astype(int)

for c in priority_places_ranked[indicator_cols].columns:
    priority_places_ranked[c] = (priority_places_ranked[c] - 0.5) / priority_places['country_denominator']
    priority_places_ranked[c] = sp.stats.norm.ppf(priority_places_ranked[c], loc=0, scale=1)

priority_places_ranked['country'] = priority_places_ranked.index.str[0]

# combine into domains
priority_places_ranked['domain_supermarket_proximity'] = 0.5 * priority_places_ranked[['supermarket_distance', 'supermarket_1kmcount']].sum(axis=1)
priority_places_ranked['domain_supermarket_accessibility'] = 0.5 * priority_places_ranked[['AccessibilityViaPublicTransport', 'AverageTravelDistance']].sum(axis=1)
priority_places_ranked['domain_ecommerce_access'] = 0.5 * priority_places_ranked[['online_propensity', 'OnlineGroceryAvailability']].sum(axis=1)
priority_places_ranked['domain_socio_demographic'] = 0.5 * priority_places_ranked[['no_car_indicator', 'inc_dep_indicator']].sum(axis=1)
priority_places_ranked['domain_nonsupermarket_proximity'] = 0.25 * priority_places_ranked[['nonsupermarket_distance','nonsupermarket_1kmcount','market_1km_count', 'market_distance']].sum(axis=1)
priority_places_ranked['domain_food_for_families'] = 0.25 * priority_places_ranked[['foodbank_distance','healthy_start_voucher_uptake','households_children_UC_percent']].sum(axis=1)
priority_places_ranked['domain_fuel_poverty'] = 0.5 * priority_places_ranked[['fuel_poverty_pct', 'prepayment_prevalence']].sum(axis=1)

domain_columns = [
    'domain_supermarket_proximity', 'domain_supermarket_accessibility',
    'domain_ecommerce_access', 'domain_socio_demographic',
    'domain_nonsupermarket_proximity', 'domain_food_for_families',
    'domain_fuel_poverty']

# rank domains
priority_places_domains = priority_places_ranked[domain_columns + ['country']].groupby('country').rank(method='min').astype(int)
priority_places_domains['country'] = priority_places_domains.index.str[0]
priority_places_domains = priority_places_domains.merge(country_counts, left_on='country', right_index=True)
priority_places_domains.rename({'index': 'country_denominator'}, axis=1, inplace=True)

# exponential transform
priority_places_domains_normalised = pd.DataFrame(columns=priority_places_domains[domain_columns].columns)
for c in domain_columns:
    priority_places_domains_normalised[c] = -23 * np.log(
        1 - (priority_places_domains[c] / priority_places_domains['country_denominator'])
        * (1 - np.exp(-100 / 23)))

# combined domain weighting
priority_places_domains['combined'] = (
    0.125 * priority_places_domains_normalised['domain_supermarket_proximity'] +
    0.125 * priority_places_domains_normalised['domain_supermarket_accessibility'] +
    0.125 * priority_places_domains_normalised['domain_ecommerce_access'] +
    0.125 * priority_places_domains_normalised['domain_nonsupermarket_proximity'] +
    (1/6) * priority_places_domains_normalised['domain_socio_demographic'] +
    (1/6) * priority_places_domains_normalised['domain_food_for_families'] +
    (1/6) * priority_places_domains_normalised['domain_fuel_poverty'])

priority_places_domains['combined'] = priority_places_domains.groupby('country')['combined'].rank(method='min').astype(int)

# compute deciles only for existing countries, as S and NI do not have LAD
priority_places_deciles = priority_places_domains.copy()

for country in priority_places_deciles['country'].unique():
    for col in domain_columns + ['combined']:
        mask = priority_places_deciles['country'] == country
        priority_places_deciles.loc[mask, col] = (
            pd.qcut(priority_places_domains.loc[mask, col],
                    10, duplicates='drop', labels=range(1, 11)).astype('int'))

# merge
priority_places_full_lad = priority_places_domains.merge(priority_places_deciles, left_index=True, right_index=True, suffixes=('', '_decile'))

# clean up duplicate columns
for col in ['country_decile', 'country_denominator_decile']:
    if col in priority_places_full_lad.columns:
        priority_places_full_lad = priority_places_full_lad.drop(columns=[col])

#renaming
priority_places_full_lad = priority_places_full_lad.rename(columns={
    'domain_supermarket_proximity_decile': 'pp_dec_domain_supermarket_proximity',
    'domain_supermarket_accessibility_decile': 'pp_dec_domain_supermarket_accessibility',
    'domain_ecommerce_access_decile': 'pp_dec_domain_ecommerce_access',
    'domain_socio_demographic_decile': 'pp_dec_domain_socio_demographic',
    'domain_nonsupermarket_proximity_decile': 'pp_dec_domain_nonsupermarket_proximity',
    'domain_food_for_families_decile': 'pp_dec_domain_food_for_families',
    'domain_fuel_poverty_decile': 'pp_dec_domain_fuel_poverty',
    'combined_decile': 'pp_dec_combined'})

In [15]:
priority_places_full_lad['domain_socio_demographic'].unique()

array([ 13,   8,  52,  82,  75,  55, 160,  26,   2,   9, 191,  43, 119,
       180,  62,  20, 290,  11, 197, 100,  40, 188,  79, 186, 242,  77,
        66, 147,  81,  73,  85, 128, 116, 269, 266,  94, 111, 265, 295,
       154,  60,  67,  76, 103,  70, 215, 164, 207, 149, 226, 233, 146,
       259, 120, 139, 217, 268, 169, 190, 124, 182, 202, 194, 115, 254,
       135, 240, 287, 159, 106,  84, 248, 131, 181, 161, 258, 228, 126,
       216, 170, 239, 195, 183, 221,  83,  29, 168, 167, 276, 114, 210,
       249, 179, 218, 173, 209, 104, 250, 238,  78, 294, 165, 281, 211,
       110, 245, 255, 229, 289, 262, 277, 140, 296, 134, 251, 177, 264,
       260, 171, 208, 196, 213, 247, 151, 178, 144, 175, 109, 121, 198,
       253, 105, 118,  48, 235, 220,  19, 193, 184,  46, 101,  64,  57,
       280, 108, 204, 138, 132, 263, 199, 291, 231, 241, 224, 192, 117,
       102,  56, 252, 201, 203, 176, 200, 273,  51, 163, 189,  44, 274,
        98, 148, 156, 158,  87, 172, 257,  96, 292, 270, 275, 13

We export the LAD-level domain/combined ranks and deciles for each lad24cd. This dataset can be joined to LAD boundaries (2024) for mapping and used for LAD-level comparisons.

In [16]:
priority_places_full_lad.to_csv('ppfi_final_lad.csv')